In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.AOint('aug-cc-pvqz', 'He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

Size of the ERI tensor will be 0.04 Gb

Ground state SCF Energy : -2.86152200 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.86152200 



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 46  
Molecular orbitals: 46  
Spin Orbitals     : 92  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 45  



In [4]:
scf_energy = aoint.scf_energy
mo_erints = mo_erints.transpose(0,2,1,3)
psi4_cisd = psi4.energy('CISD')

In [5]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = True
doubles_options = [True, False, True, True, True]
options = singles, full_cis, doubles, doubles_options
csfs, num_csfs = pyci.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)

([1, 45, 45, 0, 0, 0, 0], 91)

In [6]:
HCISD = pyci.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=22)

In [7]:
HCISD.shape

(91, 91)

In [8]:
vals, vecs = np.linalg.eigh(HCISD)

In [9]:
vals[0], psi4_cisd


(-40.293672795925005, -2.902533599417926)

In [ ]:
HCISD.shape

In [ ]:
E0, scf_energy

In [ ]:
num_csfs, sum(num_csfs)
